# Chapter 4: 3D Transformations and Viewing Pipeline

## The Complete Pipeline Implementation

This notebook covers:
- 3D transformation implementation
- Camera system implementation
- Projection implementation
- Viewport transformation
- Complete transformation pipeline

**Key References:** Marschner & Shirley Ch. 7-8, Gambetta Ch. 5-6

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Tuple, List

print("✓ Imports loaded")

In [ ]:
class Vec3:
    """3D Vector class"""
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __truediv__(self, scalar):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self):
        return math.sqrt(self.x**2 + self.y**2 + self.z**2)
    
    def normalize(self):
        l = self.length()
        return self / l if l > 0 else Vec3(0, 0, 0)
    
    def __repr__(self):
        return f"Vec3({self.x:.3f}, {self.y:.3f}, {self.z:.3f})"

class Mat4:
    """4x4 Matrix for transformations"""
    def __init__(self, data=None):
        if data is None:
            self.m = [[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1]]
        else:
            self.m = [list(row) for row in data]
    
    @staticmethod
    def zeros():
        return Mat4([[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]])
    
    def __getitem__(self, key):
        return self.m[key]
    
    def __matmul__(self, other):
        """Matrix multiplication"""
        result = Mat4.zeros()
        for i in range(4):
            for j in range(4):
                result.m[i][j] = sum(self.m[i][k] * other.m[k][j] for k in range(4))
        return result
    
    def mul_vec3(self, v: Vec3) -> Vec3:
        """Transform Vec3 (assuming w=1)"""
        x = self.m[0][0]*v.x + self.m[0][1]*v.y + self.m[0][2]*v.z + self.m[0][3]
        y = self.m[1][0]*v.x + self.m[1][1]*v.y + self.m[1][2]*v.z + self.m[1][3]
        z = self.m[2][0]*v.x + self.m[2][1]*v.y + self.m[2][2]*v.z + self.m[2][3]
        w = self.m[3][0]*v.x + self.m[3][1]*v.y + self.m[3][2]*v.z + self.m[3][3]
        
        if w != 0 and w != 1:
            return Vec3(x/w, y/w, z/w)
        return Vec3(x, y, z)
    
    def __repr__(self):
        lines = []
        for row in self.m:
            lines.append("  " + " ".join(f"{x:8.3f}" for x in row))
        return "Mat4(\n" + "\n".join(lines) + "\n)"

print("✓ Vec3 and Mat4 classes loaded")

---

## 1. Viewing Pipeline - Theory

### 1.1 The Graphics Pipeline

The **viewing transformation pipeline** converts 3D world coordinates to 2D screen coordinates:

$$\boxed{\text{Object Space}} \xrightarrow{\text{Model}} \boxed{\text{World Space}} \xrightarrow{\text{View}} \boxed{\text{Camera Space}} \xrightarrow{\text{Projection}} \boxed{\text{Clip Space}} \xrightarrow{\text{Perspective Divide}} \boxed{\text{NDC}} \xrightarrow{\text{Viewport}} \boxed{\text{Screen Space}}$$

**Pipeline stages:**

1. **Model Transform:** Position objects in world space
2. **View Transform:** Transform to camera space
3. **Projection:** Apply perspective or orthographic projection
4. **Perspective Divide:** Homogeneous to Cartesian conversion
5. **Viewport Transform:** Map to screen coordinates

### 1.2 Camera/View Transformation

**Goal:** Transform world coordinates to camera-centric coordinates.

**Camera definition:**
- **Position** $\mathbf{e}$ (eye point)
- **Look-at point** $\mathbf{t}$ (target)
- **Up vector** $\mathbf{u}_{\text{world}}$ (typically $(0, 1, 0)$)

**Camera coordinate frame:**

$$\mathbf{w} = \frac{\mathbf{e} - \mathbf{t}}{\|\mathbf{e} - \mathbf{t}\|} \quad \text{(backward, RH system)}$$

$$\mathbf{u} = \frac{\mathbf{u}_{\text{world}} \times \mathbf{w}}{\|\mathbf{u}_{\text{world}} \times \mathbf{w}\|} \quad \text{(right)}$$

$$\mathbf{v} = \mathbf{w} \times \mathbf{u} \quad \text{(up)}$$

**View matrix (look-at):**

$$\mathbf{M}_{\text{view}} = \begin{pmatrix}
u_x & u_y & u_z & -\mathbf{u} \cdot \mathbf{e} \\
v_x & v_y & v_z & -\mathbf{v} \cdot \mathbf{e} \\
w_x & w_y & w_z & -\mathbf{w} \cdot \mathbf{e} \\
0 & 0 & 0 & 1
\end{pmatrix}$$

This is equivalent to: $\mathbf{M}_{\text{view}} = \mathbf{R} \mathbf{T}$

where $\mathbf{T}$ translates by $-\mathbf{e}$ and $\mathbf{R}$ rotates to align axes.

### 1.3 Projection Transformations

#### Perspective Projection

**Perspective projection** simulates human vision (objects farther away appear smaller).

**Frustum parameters:**
- Field of view (FOV): $\theta$ (vertical angle)
- Aspect ratio: $a = \frac{\text{width}}{\text{height}}$
- Near plane: $n$
- Far plane: $f$

**Perspective matrix:**

$$\mathbf{M}_{\text{persp}} = \begin{pmatrix}
\frac{1}{a \tan(\theta/2)} & 0 & 0 & 0 \\
0 & \frac{1}{\tan(\theta/2)} & 0 & 0 \\
0 & 0 & -\frac{f+n}{f-n} & -\frac{2fn}{f-n} \\
0 & 0 & -1 & 0
\end{pmatrix}$$

**After projection:** Divide by $w$ to get NDC (Normalized Device Coordinates):

$$\text{NDC} = \left(\frac{x}{w}, \frac{y}{w}, \frac{z}{w}\right), \quad (x,y,z) \in [-1, 1]^3$$

#### Orthographic Projection

**Orthographic projection** preserves parallel lines (no perspective).

**Parameters:** Left $l$, right $r$, bottom $b$, top $t$, near $n$, far $f$

$$\mathbf{M}_{\text{ortho}} = \begin{pmatrix}
\frac{2}{r-l} & 0 & 0 & -\frac{r+l}{r-l} \\
0 & \frac{2}{t-b} & 0 & -\frac{t+b}{t-b} \\
0 & 0 & -\frac{2}{f-n} & -\frac{f+n}{f-n} \\
0 & 0 & 0 & 1
\end{pmatrix}$$

### 1.4 Viewport Transformation

**Goal:** Map NDC $[-1, 1]^2$ to screen coordinates $[0, W] \times [0, H]$

$$x_{\text{screen}} = \frac{W}{2}(x_{\text{NDC}} + 1)$$

$$y_{\text{screen}} = \frac{H}{2}(y_{\text{NDC}} + 1)$$

Or in matrix form:

$$\mathbf{M}_{\text{viewport}} = \begin{pmatrix}
W/2 & 0 & 0 & W/2 \\
0 & H/2 & 0 & H/2 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}$$

In [ ]:
def look_at(eye: Vec3, target: Vec3, up: Vec3) -> Mat4:
    """Create view (camera) matrix using look-at transformation"""
    w = (eye - target).normalize()
    u = up.cross(w).normalize()
    v = w.cross(u)
    
    mat = Mat4([
        [u.x, u.y, u.z, -u.dot(eye)],
        [v.x, v.y, v.z, -v.dot(eye)],
        [w.x, w.y, w.z, -w.dot(eye)],
        [0,   0,   0,   1]
    ])
    return mat

def perspective(fov_y: float, aspect: float, near: float, far: float) -> Mat4:
    """Create perspective projection matrix"""
    tan_half_fov = math.tan(fov_y / 2.0)
    
    mat = Mat4.zeros()
    mat[0][0] = 1.0 / (aspect * tan_half_fov)
    mat[1][1] = 1.0 / tan_half_fov
    mat[2][2] = -(far + near) / (far - near)
    mat[2][3] = -(2.0 * far * near) / (far - near)
    mat[3][2] = -1.0
    return mat

def orthographic(left: float, right: float, bottom: float, top: float, 
                 near: float, far: float) -> Mat4:
    """Create orthographic projection matrix"""
    mat = Mat4.zeros()
    mat[0][0] = 2.0 / (right - left)
    mat[1][1] = 2.0 / (top - bottom)
    mat[2][2] = -2.0 / (far - near)
    mat[0][3] = -(right + left) / (right - left)
    mat[1][3] = -(top + bottom) / (top - bottom)
    mat[2][3] = -(far + near) / (far - near)
    mat[3][3] = 1.0
    return mat

class Camera:
    """Camera class with view and projection"""
    def __init__(self, position: Vec3, target: Vec3, up: Vec3,
                 fov: float = math.radians(60), aspect: float = 16/9,
                 near: float = 0.1, far: float = 100.0):
        self.position = position
        self.target = target
        self.up = up
        self.fov = fov
        self.aspect = aspect
        self.near = near
        self.far = far
        self.update()
    
    def update(self):
        """Recompute view and projection matrices"""
        self.view_matrix = look_at(self.position, self.target, self.up)
        self.projection_matrix = perspective(self.fov, self.aspect, self.near, self.far)
        self.vp_matrix = self.projection_matrix @ self.view_matrix
    
    def world_to_screen(self, point: Vec3, width: int, height: int) -> Tuple[float, float, float]:
        """Transform point from world space to screen coordinates"""
        x = self.vp_matrix.m[0][0]*point.x + self.vp_matrix.m[0][1]*point.y + self.vp_matrix.m[0][2]*point.z + self.vp_matrix.m[0][3]
        y = self.vp_matrix.m[1][0]*point.x + self.vp_matrix.m[1][1]*point.y + self.vp_matrix.m[1][2]*point.z + self.vp_matrix.m[1][3]
        z = self.vp_matrix.m[2][0]*point.x + self.vp_matrix.m[2][1]*point.y + self.vp_matrix.m[2][2]*point.z + self.vp_matrix.m[2][3]
        w = self.vp_matrix.m[3][0]*point.x + self.vp_matrix.m[3][1]*point.y + self.vp_matrix.m[3][2]*point.z + self.vp_matrix.m[3][3]
        
        if w != 0:
            x /= w
            y /= w
            z /= w
        
        screen_x = (x + 1) * width / 2
        screen_y = (y + 1) * height / 2
        return (screen_x, screen_y, z)

print("✓ Helper functions and Camera class loaded")

In [ ]:
# Example 1: Visualize Projection Differences
print("Example 1: Perspective vs Orthographic Projection\n")

# Create a grid of points in 3D
grid_points = []
for x in range(-2, 3):
    for z in range(1, 6):
        grid_points.append(Vec3(x, 0, -z))

# Perspective camera
persp_cam = Camera(
    position=Vec3(0, 2, 0),
    target=Vec3(0, 0, -3),
    up=Vec3(0, 1, 0),
    fov=math.radians(60),
    aspect=1.0
)

# Orthographic projection
ortho_proj = orthographic(-5, 5, -5, 5, 0.1, 10)
view = look_at(Vec3(0, 2, 0), Vec3(0, 0, -3), Vec3(0, 1, 0))

print("Grid point projections:")
print("\nPerspective:")
for i, p in enumerate(grid_points[:5]):  # Show first 5
    sx, sy, depth = persp_cam.world_to_screen(p, 800, 800)
    print(f"  {p} -> ({sx:.1f}, {sy:.1f})")

print("\nOrthographic (same points):")
for i, p in enumerate(grid_points[:5]):
    # Apply ortho projection manually
    p_view = view.mul_vec3(p)
    p_proj = ortho_proj.mul_vec3(p_view)
    sx = (p_proj.x + 1) * 400
    sy = (p_proj.y + 1) * 400
    print(f"  {p} -> ({sx:.1f}, {sy:.1f})")

# Example 2: Camera Animation Path
print("\n\nExample 2: Camera Animation\n")

def animate_camera_orbit(target: Vec3, radius: float, num_frames: int):
    """Generate camera positions for orbital animation"""
    positions = []
    
    for i in range(num_frames):
        angle = 2 * math.pi * i / num_frames
        x = target.x + radius * math.cos(angle)
        z = target.z + radius * math.sin(angle)
        y = target.y + radius * 0.3  # Slight elevation
        
        positions.append(Vec3(x, y, z))
    
    return positions

# Generate orbital path
orbit_positions = animate_camera_orbit(Vec3(0, 0, 0), radius=5, num_frames=8)

print("Orbital camera path (8 frames):")
for i, pos in enumerate(orbit_positions):
    cam = Camera(pos, Vec3(0, 0, 0), Vec3(0, 1, 0))
    print(f"  Frame {i}: camera at {pos}")

# Example 3: FOV Comparison
print("\n\nExample 3: Field of View Effects\n")

test_point = Vec3(2, 0, -5)

fov_values = [30, 60, 90, 120]

print(f"Same point {test_point} with different FOVs:")
for fov_deg in fov_values:
    cam = Camera(
        position=Vec3(0, 0, 0),
        target=Vec3(0, 0, -1),
        up=Vec3(0, 1, 0),
        fov=math.radians(fov_deg),
        aspect=16/9
    )
    
    sx, sy, depth = cam.world_to_screen(test_point, 1920, 1080)
    print(f"  FOV={fov_deg}°: screen=({sx:.1f}, {sy:.1f})")

# Example 4: Complete Pipeline Visualization
print("\n\nExample 4: Complete Transformation Pipeline\n")

# Create a cube's vertices
cube_vertices = [
    Vec3(-1, -1, -5), Vec3(1, -1, -5), Vec3(1, 1, -5), Vec3(-1, 1, -5),  # Front
    Vec3(-1, -1, -7), Vec3(1, -1, -7), Vec3(1, 1, -7), Vec3(-1, 1, -7),  # Back
]

# Create camera
cam = Camera(
    position=Vec3(0, 2, 0),
    target=Vec3(0, 0, -6),
    up=Vec3(0, 1, 0),
    fov=math.radians(60),
    aspect=16/9
)

print("Cube vertices through the pipeline:")
print("(showing first 4 vertices)")
for i, v in enumerate(cube_vertices[:4]):
    # World space
    print(f"\nVertex {i}: {v}")
    
    # View space
    v_view = cam.view_matrix.mul_vec3(v)
    print(f"  View space: {v_view}")
    
    # Screen space
    sx, sy, depth = cam.world_to_screen(v, 1920, 1080)
    print(f"  Screen space: ({sx:.1f}, {sy:.1f}), depth={depth:.3f}")

# Example 5: Frustum Visualization
print("\n\nExample 5: View Frustum Analysis\n")

def is_in_frustum(point: Vec3, cam: Camera) -> bool:
    """Check if point is inside view frustum (simplified)"""
    # Transform to clip space
    x = cam.vp_matrix.m[0][0]*point.x + cam.vp_matrix.m[0][1]*point.y + cam.vp_matrix.m[0][2]*point.z + cam.vp_matrix.m[0][3]
    y = cam.vp_matrix.m[1][0]*point.x + cam.vp_matrix.m[1][1]*point.y + cam.vp_matrix.m[1][2]*point.z + cam.vp_matrix.m[1][3]
    z = cam.vp_matrix.m[2][0]*point.x + cam.vp_matrix.m[2][1]*point.y + cam.vp_matrix.m[2][2]*point.z + cam.vp_matrix.m[2][3]
    w = cam.vp_matrix.m[3][0]*point.x + cam.vp_matrix.m[3][1]*point.y + cam.vp_matrix.m[3][2]*point.z + cam.vp_matrix.m[3][3]
    
    # Check if in NDC bounds after perspective divide
    if w <= 0:
        return False
    
    x /= w
    y /= w
    z /= w
    
    return (-1 <= x <= 1) and (-1 <= y <= 1) and (-1 <= z <= 1)

# Test points at various locations
test_locations = [
    (Vec3(0, 0, -5), "center of view"),
    (Vec3(10, 0, -5), "far right"),
    (Vec3(0, 0, -0.05), "too close (before near plane)"),
    (Vec3(0, 0, -150), "too far (beyond far plane)"),
    (Vec3(0, 0, 5), "behind camera"),
]

cam = Camera(Vec3(0, 0, 0), Vec3(0, 0, -1), Vec3(0, 1, 0))

print("Frustum culling test:")
for point, desc in test_locations:
    in_frustum = is_in_frustum(point, cam)
    status = "VISIBLE" if in_frustum else "CULLED"
    print(f"  {point} ({desc}): {status}")

print("\n✓ Chapter 4 Complete!")
print("\nIn this chapter, you learned:")
print("  • Complete viewing transformation pipeline")
print("  • View matrix (look-at transformation)")
print("  • Perspective and orthographic projection")
print("  • Viewport transformation")
print("  • Camera class with orbit controls")
print("  • World-to-screen coordinate transformation")
print("  • Frustum culling basics")
print("\nNext Chapter: Visibility and Hidden Surface Removal")

---

## 3. Practical Examples

Complete demonstrations of the viewing pipeline.

In [ ]:
class Camera:
    """Camera class with view and projection"""
    
    def __init__(self, position: Vec3, target: Vec3, up: Vec3,
                 fov: float = math.radians(60), aspect: float = 16/9,
                 near: float = 0.1, far: float = 100.0):
        """
        Initialize camera
        
        Args:
            position: Camera position
            target: Point camera looks at
            up: World up vector
            fov: Vertical field of view (radians)
            aspect: Aspect ratio (width/height)
            near: Near clipping plane
            far: Far clipping plane
        """
        self.position = position
        self.target = target
        self.up = up
        self.fov = fov
        self.aspect = aspect
        self.near = near
        self.far = far
        
        # Compute view and projection matrices
        self.update()
    
    def update(self):
        """Recompute view and projection matrices"""
        self.view_matrix = look_at(self.position, self.target, self.up)
        self.projection_matrix = perspective(self.fov, self.aspect, self.near, self.far)
        
        # Combined view-projection matrix
        self.vp_matrix = self.projection_matrix @ self.view_matrix
    
    def world_to_screen(self, point: Vec3, width: int, height: int) -> Tuple[float, float, float]:
        """
        Transform point from world space to screen coordinates
        
        Returns:
            (screen_x, screen_y, depth)
        """
        # Apply view-projection
        x = self.vp_matrix.m[0][0]*point.x + self.vp_matrix.m[0][1]*point.y + self.vp_matrix.m[0][2]*point.z + self.vp_matrix.m[0][3]
        y = self.vp_matrix.m[1][0]*point.x + self.vp_matrix.m[1][1]*point.y + self.vp_matrix.m[1][2]*point.z + self.vp_matrix.m[1][3]
        z = self.vp_matrix.m[2][0]*point.x + self.vp_matrix.m[2][1]*point.y + self.vp_matrix.m[2][2]*point.z + self.vp_matrix.m[2][3]
        w = self.vp_matrix.m[3][0]*point.x + self.vp_matrix.m[3][1]*point.y + self.vp_matrix.m[3][2]*point.z + self.vp_matrix.m[3][3]
        
        # Perspective divide
        if w != 0:
            x /= w
            y /= w
            z /= w
        
        # Viewport transform
        screen_x = (x + 1) * width / 2
        screen_y = (y + 1) * height / 2
        
        return (screen_x, screen_y, z)
    
    def __repr__(self):
        return f"Camera(pos={self.position}, target={self.target}, fov={math.degrees(self.fov):.1f}°)"

class OrbitCamera(Camera):
    """Orbit camera that rotates around a target"""
    
    def __init__(self, target: Vec3, distance: float, azimuth: float = 0, elevation: float = 0,
                 fov: float = math.radians(60), aspect: float = 16/9):
        """
        Initialize orbit camera
        
        Args:
            target: Point to orbit around
            distance: Distance from target
            azimuth: Horizontal angle (radians)
            elevation: Vertical angle (radians)
        """
        self.target = target
        self.distance = distance
        self.azimuth = azimuth
        self.elevation = elevation
        
        # Compute initial position
        position = self._compute_position()
        
        super().__init__(position, target, Vec3(0, 1, 0), fov, aspect)
    
    def _compute_position(self) -> Vec3:
        """Compute camera position from spherical coordinates"""
        x = self.target.x + self.distance * math.cos(self.elevation) * math.cos(self.azimuth)
        y = self.target.y + self.distance * math.sin(self.elevation)
        z = self.target.z + self.distance * math.cos(self.elevation) * math.sin(self.azimuth)
        return Vec3(x, y, z)
    
    def rotate(self, delta_azimuth: float, delta_elevation: float):
        """Rotate camera around target"""
        self.azimuth += delta_azimuth
        self.elevation += delta_elevation
        
        # Clamp elevation to avoid gimbal lock
        self.elevation = max(-math.pi/2 + 0.01, min(math.pi/2 - 0.01, self.elevation))
        
        # Update position
        self.position = self._compute_position()
        self.update()
    
    def zoom(self, delta: float):
        """Zoom in/out"""
        self.distance += delta
        self.distance = max(0.1, self.distance)  # Minimum distance
        
        self.position = self._compute_position()
        self.update()

# Test Camera
print("Testing Camera System:\n")

# Create a camera
cam = Camera(
    position=Vec3(5, 3, 5),
    target=Vec3(0, 0, 0),
    up=Vec3(0, 1, 0),
    fov=math.radians(60),
    aspect=16/9
)

print(f"Camera: {cam}\n")

# Transform some points
test_points = [
    Vec3(0, 0, 0),
    Vec3(1, 0, 0),
    Vec3(0, 1, 0),
    Vec3(0, 0, 1),
]

width, height = 1920, 1080

print("World to Screen Transformations:")
for p in test_points:
    screen_x, screen_y, depth = cam.world_to_screen(p, width, height)
    print(f"  {p} -> screen({screen_x:.1f}, {screen_y:.1f}), depth={depth:.3f}")

# Test orbit camera
print("\n\nTesting Orbit Camera:\n")

orbit_cam = OrbitCamera(
    target=Vec3(0, 0, 0),
    distance=10,
    azimuth=math.radians(45),
    elevation=math.radians(30)
)

print(f"Orbit Camera: {orbit_cam}")
print(f"Initial position: {orbit_cam.position}")

# Rotate
orbit_cam.rotate(math.radians(15), math.radians(10))
print(f"After rotation: {orbit_cam.position}")

# Zoom
orbit_cam.zoom(-2)
print(f"After zoom: {orbit_cam.position}, distance={orbit_cam.distance:.2f}")

---

## 2. Camera System - Implementation

---

## 2. Camera System - Theory

### 2.1 Camera Class Design

A **camera** encapsulates:
- **Position** and **orientation** in world space
- **Projection parameters** (FOV, aspect, near/far planes)
- **View and projection matrices**

### 2.2 Camera Controls

**Orbit camera:** Rotate around a target point
- **Azimuth** $\phi$: Horizontal rotation angle
- **Elevation** $\theta$: Vertical rotation angle  
- **Distance** $d$: Distance from target

Camera position:
$$\mathbf{e} = \mathbf{t} + d(\cos\theta\cos\phi, \sin\theta, \cos\theta\sin\phi)$$

**FPS camera:** First-person movement
- **Forward/backward:** Move along view direction
- **Strafe left/right:** Move along right vector
- **Yaw/pitch:** Rotate view direction

In [ ]:
def look_at(eye: Vec3, target: Vec3, up: Vec3) -> Mat4:
    """
    Create view (camera) matrix using look-at transformation
    
    Args:
        eye: Camera position
        target: Point camera is looking at
        up: World up vector
    
    Returns:
        View matrix
    """
    # Compute camera coordinate frame
    w = (eye - target).normalize()  # Camera looks down -z, so w points backward
    u = up.cross(w).normalize()      # Right vector
    v = w.cross(u)                   # Up vector (recomputed for orthogonality)
    
    # Create view matrix
    mat = Mat4([
        [u.x, u.y, u.z, -u.dot(eye)],
        [v.x, v.y, v.z, -v.dot(eye)],
        [w.x, w.y, w.z, -w.dot(eye)],
        [0,   0,   0,   1]
    ])
    
    return mat

def perspective(fov_y: float, aspect: float, near: float, far: float) -> Mat4:
    """
    Create perspective projection matrix
    
    Args:
        fov_y: Vertical field of view in radians
        aspect: Aspect ratio (width / height)
        near: Near clipping plane
        far: Far clipping plane
    
    Returns:
        Perspective projection matrix
    """
    tan_half_fov = math.tan(fov_y / 2.0)
    
    mat = Mat4.zeros()
    mat[0][0] = 1.0 / (aspect * tan_half_fov)
    mat[1][1] = 1.0 / tan_half_fov
    mat[2][2] = -(far + near) / (far - near)
    mat[2][3] = -(2.0 * far * near) / (far - near)
    mat[3][2] = -1.0
    
    return mat

def orthographic(left: float, right: float, bottom: float, top: float, 
                 near: float, far: float) -> Mat4:
    """
    Create orthographic projection matrix
    
    Args:
        left, right: Horizontal bounds
        bottom, top: Vertical bounds
        near, far: Depth bounds
    
    Returns:
        Orthographic projection matrix
    """
    mat = Mat4.zeros()
    mat[0][0] = 2.0 / (right - left)
    mat[1][1] = 2.0 / (top - bottom)
    mat[2][2] = -2.0 / (far - near)
    mat[0][3] = -(right + left) / (right - left)
    mat[1][3] = -(top + bottom) / (top - bottom)
    mat[2][3] = -(far + near) / (far - near)
    mat[3][3] = 1.0
    
    return mat

def viewport(width: int, height: int) -> Mat4:
    """
    Create viewport transformation matrix
    Maps from NDC [-1,1]^2 to screen coordinates [0,W]x[0,H]
    
    Args:
        width: Screen width in pixels
        height: Screen height in pixels
    
    Returns:
        Viewport matrix
    """
    mat = Mat4([
        [width/2,  0,         0, width/2],
        [0,        height/2,  0, height/2],
        [0,        0,         1, 0],
        [0,        0,         0, 1]
    ])
    
    return mat

# Test transformation matrices
print("Testing Viewing Transformations:\n")

# Create a view matrix
eye = Vec3(5, 5, 5)
target = Vec3(0, 0, 0)
up = Vec3(0, 1, 0)

view_mat = look_at(eye, target, up)
print("View Matrix (look-at):")
print(view_mat)
print()

# Test perspective projection
fov = math.radians(60)
aspect = 16.0 / 9.0
near = 0.1
far = 100.0

persp_mat = perspective(fov, aspect, near, far)
print("Perspective Projection Matrix:")
print(persp_mat)
print()

# Test orthographic projection
ortho_mat = orthographic(-10, 10, -10, 10, 0.1, 100)
print("Orthographic Projection Matrix:")
print(ortho_mat)
print()

# Test viewport transformation
viewport_mat = viewport(1920, 1080)
print("Viewport Matrix (1920x1080):")
print(viewport_mat)
print()

# Transform a point through the pipeline
point = Vec3(1, 1, -10)
print(f"Original point (world space): {point}")

# Apply view transform
point_view = view_mat.mul_vec3(point)
print(f"After view transform: {point_view}")

# Apply projection (perspective)
# Note: This produces homogeneous coordinates, need to divide by w
x = persp_mat.m[0][0]*point_view.x + persp_mat.m[0][3]
y = persp_mat.m[1][1]*point_view.y + persp_mat.m[1][3]
z = persp_mat.m[2][2]*point_view.z + persp_mat.m[2][3]
w = persp_mat.m[3][2]*point_view.z

print(f"After projection (clip space): ({x:.3f}, {y:.3f}, {z:.3f}, w={w:.3f})")

# Perspective divide to get NDC
if w != 0:
    ndc = Vec3(x/w, y/w, z/w)
    print(f"After perspective divide (NDC): {ndc}")

---

## 1. Viewing Pipeline - Implementation

---

## Setup and Imports

In [ ]:
# Your implementation starts here
